In [1]:
import pandas as pd
import zipfile
from os import listdir
from os.path import join

In [3]:
ameri_path = '/home/aleksei/Carbon/data/AmeriFlux'
flux_path = '/home/aleksei/Carbon/data/FLUXNET2015'

ameri_files = listdir(ameri_path)
flux_files = listdir(flux_path)

In [14]:
for f in ameri_files:
    i=0
    with zipfile.ZipFile(join(ameri_path, f), 'r') as zip_ref:
        new_name = '_'.join(f.split('_')[:2])
        zip_ref.extractall(join(ameri_path, new_name))
    i+=1
print(f"{i+1} Ameri archives unzipped")
for f in flux_files:
    i=0
    with zipfile.ZipFile(join(flux_path, f), 'r') as zip_ref:
        new_name = '_'.join(f.split('_')[:2])
        zip_ref.extractall(join(flux_path, new_name))
    i+=1
print(f"{i+1} FLUXNET2015 archives unzipped")

2 Ameri archives unzipped
2 FLUXNET2015 archives unzipped


In [4]:
mode = 'LE'

In [5]:
flux_meta = 'FLX_AA-Flx'
if mode=='NEE':
    cols = ['TIMESTAMP', 'NEE_VUT_REF', 'NEE_VUT_REF_QC']
    QC = 'NEE_VUT_REF_QC'
elif mode=='LE':
    cols = ['TIMESTAMP', 'LE_F_MDS', 'LE_F_MDS_QC']
    QC = 'LE_F_MDS_QC'
flux_dir = [x for x in listdir(flux_path) if (x.split('.')[-1]!='zip')&(x!=flux_meta)]
ameri_dir = [x for x in listdir(ameri_path) if (x.split('.')[-1]!='zip')]

In [22]:
for folder in flux_dir:
    csv = [x for x in listdir(join(flux_path, folder)) if (x.split('_')[3]=='FULLSET')&(x.split('_')[4]=='DD')][0]
    df = pd.read_csv(join(flux_path, folder, csv))[cols]
    df = df[df[QC]>=0.9]
    df['site'] = ['_'.join(csv.split('_')[:2])]*len(df)
    if 'data' not in locals():
        data = df.copy()
    else:
        data=pd.concat([data,df], axis=0)
data = data.reset_index(drop=True)
data.to_csv(f'./data/all_fluxnet_{mode}.csv', index=None)

for folder in ameri_dir:
    csv = [x for x in listdir(join(ameri_path, folder)) if (x.split('_')[3]=='FULLSET')&(x.split('_')[4]=='DD')][0]
    df = pd.read_csv(join(ameri_path, folder, csv))[cols]
    df = df[df[QC]>=0.9]
    df['site'] = ['_'.join(csv.split('_')[:2])]*len(df)
    if 'data_ameri' not in locals():
        data_ameri = df.copy()
    else:
        data_ameri=pd.concat([data_ameri,df], axis=0)
data_ameri = data_ameri.reset_index(drop=True)
data_ameri.to_csv(f'./data/all_ameri_{mode}.csv', index=None)

all_data = pd.concat([data, data_ameri], axis=0).reset_index(drop=True)
all_data.to_csv(f'./data/all_fluxes_{mode}.csv', index=None)